In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import numpy as np
import pandas as pd
import os
import random
import pickle
import math
import missingno as msno
from multiprocessing.dummy import Pool as ThreadPool
import time
import sys
import csv
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as tfunc
from torch.utils.data import Dataset
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as func
import sklearn.metrics as metrics
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm import tqdm

In [ ]:
from google.colab import auth, drive
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix, f1_score, classification_report
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
auth.authenticate_user()

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files

# This will prompt you to select and upload the file manually
uploaded = files.upload()


Saving 11_final_cohort_alldata.csv.gz to 11_final_cohort_alldata.csv (1).gz


# Read in Final Cohort Details

In [ ]:
# Load the uploaded gzipped CSV file
df = pd.read_csv('11_final_cohort_alldata.csv.gz', compression='gzip')

# Display the first few rows
df.head()


,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text,admityear,admitmonth,...,adrd,ad,case_status,adrd_status,Stroke_History,Myocardial_Infarction,Peripheral_Vascular_Disease,Cerebrovascular_Disease,Diabetes_Mellitus,Cancer
0,10000980-DS-20,10000980,29654838.0,DS,20,2188-01-05,2188-01-06 20:49:00,\nName: ___ Unit No: ___\n \nAdmi...,2188,1,...,0,0,0,0,0,1,1,0,1,0
1,10000980-DS-21,10000980,26913865.0,DS,21,2189-07-03,2189-07-03 19:50:00,\nName: ___ Unit No: ___\n \nAdmi...,2189,6,...,0,0,0,0,0,1,1,0,1,0
2,10000980-DS-22,10000980,24947999.0,DS,22,2190-11-08,2190-11-09 13:57:00,\nName: ___ Unit No: ___\n \nAdmi...,2190,11,...,0,0,0,0,0,1,1,0,1,0
3,10000980-DS-23,10000980,25242409.0,DS,23,2191-04-11,2191-04-11 17:48:00,\nName: ___ Unit No: ___\n \nAdmi...,2191,4,...,0,0,0,0,0,1,1,0,1,0
4,10000980-DS-24,10000980,25911675.0,DS,24,2191-05-24,2191-05-24 17:29:00,\nName: ___ Unit No: ___\n \nAdmi...,2191,5,...,0,0,0,0,0,1,1,0,1,0


In [ ]:
# Sort the entire DataFrame by subject_id and note_seq
df_sorted = df.sort_values(by=['subject_id', 'note_seq'])

# Optional: reset index if needed
df_sorted = df_sorted.reset_index(drop=True)

# Preview sorted data
df_sorted.head()


,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text,admityear,admitmonth,...,adrd,ad,case_status,adrd_status,Stroke_History,Myocardial_Infarction,Peripheral_Vascular_Disease,Cerebrovascular_Disease,Diabetes_Mellitus,Cancer
0,10000980-DS-20,10000980,29654838.0,DS,20,2188-01-05,2188-01-06 20:49:00,\nName: ___ Unit No: ___\n \nAdmi...,2188,1,...,0,0,0,0,0,1,1,0,1,0
1,10000980-DS-21,10000980,26913865.0,DS,21,2189-07-03,2189-07-03 19:50:00,\nName: ___ Unit No: ___\n \nAdmi...,2189,6,...,0,0,0,0,0,1,1,0,1,0
2,10000980-DS-22,10000980,24947999.0,DS,22,2190-11-08,2190-11-09 13:57:00,\nName: ___ Unit No: ___\n \nAdmi...,2190,11,...,0,0,0,0,0,1,1,0,1,0
3,10000980-DS-23,10000980,25242409.0,DS,23,2191-04-11,2191-04-11 17:48:00,\nName: ___ Unit No: ___\n \nAdmi...,2191,4,...,0,0,0,0,0,1,1,0,1,0
4,10000980-DS-24,10000980,25911675.0,DS,24,2191-05-24,2191-05-24 17:29:00,\nName: ___ Unit No: ___\n \nAdmi...,2191,5,...,0,0,0,0,0,1,1,0,1,0


In [ ]:
# First, separate out the 'text' field
text_concat = df_sorted.groupby('subject_id')['text'].apply(lambda x: ' \n New Note:\n '.join(x)).rename('text')

# Then, take the max of all other columns (excluding 'text')
other_max = df_sorted.drop(columns='text').groupby('subject_id').max()

# Combine both into one DataFrame
df_grouped = pd.concat([other_max, text_concat], axis=1)

# View result
df_grouped.head()


,note_id,hadm_id,note_type,note_seq,charttime,storetime,admityear,admitmonth,admitday,admittime,...,ad,case_status,adrd_status,Stroke_History,Myocardial_Infarction,Peripheral_Vascular_Disease,Cerebrovascular_Disease,Diabetes_Mellitus,Cancer,text
subject_id,,,,,,,,,,,,,,,,,,,,,
10000980,10000980-RR-87,29659838.0,RR,87,2193-08-17,2193-08-17 16:05:00,2193,11,27,20:57:00,...,0,0,0,0,1,1,0,1,0,\nName: ___ Unit No: ___\n \nAdmi...
10001401,10001401-RR-49,28058085.0,RR,49,2133-07-13 18:00:00,2133-07-14 11:42:00,2133,11,30,23:15:00,...,0,0,0,0,0,0,0,0,1,INDICATION: ___ year old woman status post ra...
10001667,10001667-RR-21,22672901.0,RR,21,2173-08-24,2173-08-24 15:04:00,2173,8,22,17:16:00,...,0,0,0,0,0,0,0,0,0,\nName: ___ Unit No: ___\n ...
10001860,10001860-RR-22,21441082.0,RR,22,2188-03-30,2188-03-29 12:56:00,2188,3,27,01:54:00,...,0,0,0,0,0,0,0,0,0,\nName: ___ Unit No: ___...
10002131,10002131-RR-20,27411540.0,RR,20,2128-03-19,2128-03-19 16:38:00,2128,4,17,14:53:00,...,1,1,1,0,0,0,0,0,0,"INDICATION: ___ woman with nausea, vomiting, ..."


In [ ]:
df_grouped.rename(columns = {'text': 'TEXT'}, inplace = True)

In [ ]:
df_grouped['TEXT']

,TEXT
subject_id,
10000980,\nName: ___ Unit No: ___\n \nAdmi...
10001401,INDICATION: ___ year old woman status post ra...
10001667,\nName: ___ Unit No: ___\n ...
10001860,\nName: ___ Unit No: ___...
10002131,"INDICATION: ___ woman with nausea, vomiting, ..."
...,...
19970466,INDICATION: ___ year old woman pre-op for TEV...
19971409,EXAMINATION: MR ___ SCAN WITHOUT CONTRAST ___...
19971844,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...


In [ ]:
#df = pd.read_csv("trimmed_dataset_period.csv")

In [ ]:
#df.rename(columns = {'text': 'TEXT'}, inplace = True)

In [ ]:
#df.dropna(inplace = True)

# Summarize concatenated notes with Gemini

In [ ]:
import vertexai
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Image,
    Part,
    SafetySetting,
)
from vertexai.batch_prediction import BatchPredictionJob
import json
from google.cloud import storage

# replace with project ID from Google Cloud Platform
PROJECT_ID = "mlhealthcare2"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
def df_to_jsonl_gcs(df, bucket_name, blob_name):
    """Converts a DataFrame to JSONL and uploads to Google Cloud Storage.

    Args:
        df: Pandas DataFrame with a 'text' column.
        bucket_name: Name of your Google Cloud Storage bucket.
        blob_name: Desired name for the JSONL file on GCS.
    """

    # Initialize a GCS client
    storage_client = storage.Client(project=PROJECT_ID)
    bucket = storage_client.bucket(bucket_name)
    if not bucket.exists():
        bucket.create(location='US')
        print(f'Bucket {bucket_name} created.')
    else:
        print(f'Bucket {bucket_name} already exists.')

    blob = bucket.blob(blob_name)

    # Write JSONL data to a string buffer
    jsonl_data = ""
    for index, row in df.iterrows():
      if pd.isnull(row['TEXT']) or not isinstance(row['TEXT'], str):
          continue  # skip if text is missing or not a string

      text = row['TEXT'][:50000]

      prompt = f"""Please mask any mention of Alzheimer's Disease or AD and any medications used to directly treat
      Alzheimer's Disease. Then, provide a clear summary of each note: {text}"""

      json_data = {
          "id": index,
          "request": {
              "contents": [
                  {
                      "role": "user",
                      "parts": [{"text": prompt}]
                  }
              ],
              "generationConfig": {"temperature": 0.4, "maxOutputTokens": 512},
          }
      }
      jsonl_data += json.dumps(json_data) + '\n'


    # Upload the JSONL data to GCS
    blob.upload_from_string(jsonl_data, content_type='application/jsonl')
    print(f"JSONL file uploaded to gs://{bucket_name}/{blob_name}")

    return f"gs://{bucket_name}/{blob_name}"

In [ ]:
# Specify a custom bucket name
BUCKET_NAME = 'health_bucket60'

input_uri = df_to_jsonl_gcs(df_grouped, BUCKET_NAME, 'gemini_batch_requests.jsonl')

Bucket health_bucket60 created.
JSONL file uploaded to gs://health_bucket60/gemini_batch_requests.jsonl


In [ ]:
output_uri = f"gs://{BUCKET_NAME}/batch-prediction/"

# Submit a batch prediction job with Gemini model
batch_prediction_job = BatchPredictionJob.submit(
    source_model="gemini-1.5-flash-001",
    input_dataset=input_uri,
    output_uri_prefix=output_uri,
)

# Check job status
print(f"Job resource name: {batch_prediction_job.resource_name}")
print(f"Model resource name with the job: {batch_prediction_job.model_name}")
print(f"Job state: {batch_prediction_job.state.name}")

# Refresh the job until complete
while not batch_prediction_job.has_ended:
    time.sleep(5)
    batch_prediction_job.refresh()

# Check if the job succeeds
if batch_prediction_job.has_succeeded:
    print("Job succeeded!")
else:
    print(f"Job failed: {batch_prediction_job.error}")

# Check the location of the output
print(f"Job output location: {batch_prediction_job.output_location}")

# Example response:
#  Job output location: gs://your-bucket/gen-ai-batch-prediction/prediction-model-year-month-day-hour:minute:second.12345

INFO:vertexai.batch_prediction._batch_prediction:BatchPredictionJob created. Resource name: projects/615148075900/locations/us-central1/batchPredictionJobs/5027004700206039040
INFO:vertexai.batch_prediction._batch_prediction:To use this BatchPredictionJob in another session:
INFO:vertexai.batch_prediction._batch_prediction:job = batch_prediction.BatchPredictionJob('projects/615148075900/locations/us-central1/batchPredictionJobs/5027004700206039040')
INFO:vertexai.batch_prediction._batch_prediction:View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5027004700206039040?project=615148075900


Job resource name: projects/615148075900/locations/us-central1/batchPredictionJobs/5027004700206039040
Model resource name with the job: publishers/google/models/gemini-1.5-flash-001
Job state: JOB_STATE_PENDING
Job succeeded!
Job output location: gs://health_bucket60/batch-prediction/prediction-model-2025-04-21T17:51:55.980302Z


In [ ]:
# Load the JSONL file into a DataFrame
# once you've made your predictions, they should be
# stored at your google cloud storage bucket specified by the path
# and you should be able to download it from this path
output_path = 'gs://health_bucket60/batch-prediction/prediction-model-2025-04-21T17:51:55.980302Z'

predictions_df = pd.read_json(output_path, lines=True)
# predictions_df = predictions_df.join(pd.json_normalize(predictions_df["response"], "candidates"))


In [ ]:
from google.cloud import storage

bucket_name = 'health_bucket60'

# ✅ Remove 'gs://' — this should be the path inside the bucket only
blob_path = 'batch-prediction/prediction-model-2025-04-21T17:51:55.980302Z/predictions.jsonl'
destination_file = 'predictions.jsonl'

client = storage.Client(project=PROJECT_ID)
bucket = client.bucket(bucket_name)
blob = bucket.blob(blob_path)
blob.download_to_filename(destination_file)

print(f"✅ Downloaded predictions to {destination_file}")


✅ Downloaded predictions to predictions.jsonl


In [ ]:
with open("predictions.jsonl", "r") as f:
    data = [json.loads(line) for line in f]

print(f"Total summaries received: {len(data)}")


Total summaries received: 2176


In [ ]:
import json

with open("predictions.jsonl", "r") as f:
    for i, line in enumerate(f):
        parsed = json.loads(line)
        print(f"\n--- Entry {i} ---")
        print("Top-level keys:", parsed.keys())
        print(json.dumps(parsed, indent=2))
        if i >= 2: break  # Only print first 3 lines



--- Entry 0 ---
Top-level keys: dict_keys(['id', 'status', 'processed_time', 'request', 'response'])
{
  "id": "17602334",
  "status": "",
  "processed_time": "2025-04-21T17:54:07.329+00:00",
  "request": {
    "contents": [
      {
        "parts": [
          {
            "text": "Please mask any mention of Alzheimer's Disease or AD and any medications used to directly treat \n      Alzheimer's Disease. Then, provide a clear summary of each note:  \nName:  ___                        Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   M\n \nService: MEDICINE\n \nAllergies: \nPenicillins\n \nAttending: ___.\n \nChief Complaint:\nLEAVING AGAINST MEDICAL ADVICE\n\nwrist pain\n \nMajor Surgical or Invasive Procedure:\nLEAVING AGAINST MEDICAL ADVICE\n\nnone\n \nHistory of Present Illness:\nLEAVING AGAINST MEDICAL ADVICE\n\n___ w/ IDDM, gout, CKD thought to be ___ DM, urethral mass, p/w \nleft arm/wrist pain as well as ongo

In [ ]:
import pandas as pd
import json

# Step 1: Read JSONL file
with open("predictions.jsonl", "r") as f:
    data = [json.loads(line) for line in f]

# Step 2: Extract summaries
results = []
for entry in data:
    summary_text = (
        entry.get("response", {})
             .get("candidates", [{}])[0]
             .get("content", {})
             .get("parts", [{}])[0]
             .get("text", "")
    )
    results.append({
        "id": entry.get("id"),
        "summary": summary_text
    })

# Step 3: Create DataFrame
summaries_df = pd.DataFrame(results)

# View result
summaries_df.head()


,id,summary
0,17602334,## Patient Note Summaries:\n\n**Note 1:**\n\nT...
1,10063534,## Patient Note Summaries:\n\n**Note 1:**\n\n*...
2,10282002,## Summary of Patient Notes:\n\n**Note 1:**\n...
3,10061468,"## Note 1 Summary:\n\n**Patient:** Female, pre..."
4,12571982,## CT ABDOMEN AND PELVIS WITH CONTRAST\n\n**Su...


In [ ]:
summaries_df.shape

(2176, 2)

In [ ]:
summaries_df

,summary
id,
17602334,## Patient Note Summaries:\n\n**Note 1:**\n\nT...
10063534,## Patient Note Summaries:\n\n**Note 1:**\n\n*...
10282002,## Summary of Patient Notes:\n\n**Note 1:**\n...
10061468,"## Note 1 Summary:\n\n**Patient:** Female, pre..."
12571982,## CT ABDOMEN AND PELVIS WITH CONTRAST\n\n**Su...
...,...
10259262,## Patient Note Summaries:\n\n**Admission Not...
13109130,## Note Summaries:\n\n**Note 1:**\n\n**Indicat...
13129811,## Patient Note Summaries:\n\n**Admission Not...


In [ ]:
# Make sure both indexes are int for proper alignment
summaries_df.index = summaries_df.index.astype(int)
df_grouped.index = df_grouped.index.astype(int)

# Join summaries based on subject_id index
df_grouped['gemini_summary'] = summaries_df['summary']

# Preview
df_grouped[['gemini_summary']].dropna().head()


,gemini_summary
subject_id,
10000980,## Note 1 Summary:\n\nThis note is a medical r...
10001401,## Summary of Medical Notes:\n\n**Note 1:**\n\...
10001667,## Patient Note Summaries:\n\n**Admission Not...
10001860,## Summary of Notes:\n\n**Note 1:** \n- **Pat...
10002131,## Note Summaries:\n\n**Note 1:**\n\n* **Indic...


In [ ]:
# Drop TEXT field
df_grouped.drop(columns = {'TEXT'}, inplace = True)

In [ ]:
# Save to CSV in Colab's local environment
df_grouped.to_csv("df_grouped_with_summaries.csv")

from google.colab import files

# Trigger the download
files.download("df_grouped_with_summaries.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>